<a href="https://colab.research.google.com/github/Misudhari/Deep-Learning-Lab/blob/main/DL_Assign_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name !='/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
import pandas as pd
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Dataset/train_a2.csv")

In [ ]:
data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
795,796,0,2,"Otter, Mr. Richard",male,39.0,0,0,28213,13.0000,NaN,S
876,877,0,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S
44,45,1,3,"Devaney, Miss. Margaret Delia",female,19.0,0,0,330958,7.8792,NaN,Q
388,389,0,3,"Sadlier, Mr. Matthew",male,NaN,0,0,367655,7.7292,NaN,Q
56,57,1,2,"Rugg, Miss. Emily",female,21.0,0,0,C.A. 31026,10.5000,NaN,S


In [ ]:
data.shape

(891, 12)

In [ ]:
#droping the unnecessary column
data.drop(columns = ['Name','Ticket','Cabin'],inplace = True)
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [ ]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [ ]:
for column in ['Age','Embarked']:
    data[column].fillna(data[column].mode()[0], inplace=True)
data['Age']=data['Age'].astype('int64')

In [ ]:
from sklearn.preprocessing import LabelEncoder
la = LabelEncoder()

def cat_to_num(c,data): 
  for i in range(len(c)):
    data.iloc[:,c[i]] = la.fit_transform(data.iloc[:,c[i]])
    #print('\nSkewness for Column: ',skew(data.iloc[:,c[i]]))
  return data.sample(3)

In [ ]:
cat_to_num([3],data)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
21,22,1,2,1,34,0,0,13.00,S
587,588,1,1,1,60,1,1,79.20,C
304,305,0,3,1,24,0,0,8.05,S


In [ ]:
cat_to_num([8],data)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
678,679,0,3,0,43,1,6,46.9000,2
309,310,1,1,0,30,0,0,56.9292,0
620,621,0,3,1,27,1,0,14.4542,0


In [ ]:
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [ ]:
data.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.647587,28.551066,0.523008,0.381594,32.204208,1.536476
std,257.353842,0.486592,0.836071,0.477990,13.207013,1.102743,0.806057,49.693429,0.791503
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,1.000000,24.000000,0.000000,0.000000,14.454200,2.000000
75%,668.500000,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [ ]:
x = data.drop(['Survived'], axis=1)
y = data.iloc[:,1].values
#y=data('Survived')

In [ ]:
x.shape

(891, 8)

In [ ]:
y.shape

(891,)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mx = MinMaxScaler()

In [ ]:
x = mx.fit_transform(x)
y= mx.fit_transform(y.reshape(-1,1))

In [ ]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
    model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value=32,
    max_value=512,step=32),
    activation='relu'))
  model.add(layers.Dense(1, activation='linear'))

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
                loss='mean_absolute_error',
                metrics=['mean_absolute_error'])
  return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Power_Consumption')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x,y, epochs=5 , validation_split=0.2)

Trial 5 Complete [00h 00m 06s]
val_mean_absolute_error: 0.19705977539221445

Best val_mean_absolute_error So Far: 0.19705977539221445
Total elapsed time: 00h 00m 33s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in project1/Power_Consumption
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 15
units_0: 480
units_1: 128
learning_rate: 0.001
units_2: 96
units_3: 64
units_4: 128
units_5: 384
units_6: 352
units_7: 96
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
Score: 0.19705977539221445
Trial summary
Hyperparameters:
num_layers: 4
units_0: 32
units_1: 288
learning_rate: 0.01
units_2: 480
units_3: 96
units_4: 512
units_5: 224
units_6: 96
units_7: 448
Score: 0.1991890867551168
Trial summary
Hyperparameters:
num_layers: 8
units_0: 192
units_1: 384
learning_rate: 0.001
units_2: 384
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
Score: 0.2003020147482554
Trial summary
Hyperparameters:
num_layers: 3
units_0: 96
units_1: 480
learning_rate: 0.0001
units_2: 32
Score: 0.29201897978782654
Trial summary
Hyperparameters:
num_layers: 8
units_0: 288
units_1: 384
learning_rate: 0.01
units_2: 288
units_3: 320
units_4: